<a href="https://colab.research.google.com/github/flintbekkers/3/blob/main/deepNN_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Application: cat or not?

**After this assignment you will be able to:**
- Build and apply a deep neural network to supervised learning. 
- Learn how to do data normalization
- learn how to do weight initialization
- Learn how to use regularization

Let's get started!

## 1 - Packages

**Exercise**: Please mount your Google drive, and set up your working folder here.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
# start your code here
os.chdir("/content/drive/MyDrive/Deep Learning Homeworks/Homework3 (1).zip (Unzipped Files)/Homework3") # change your working folder here
# end your code here

Let's first import all the packages that you will need during this assignment. 
- [numpy](https://www.numpy.org/) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a library to plot graphs in Python.
- [h5py](http://www.h5py.org) is a common package to interact with a dataset that is stored on an H5 file.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) are used here to test your model with your own picture at the end.
- dnn_app_utils provides the customized functions that will be used in this notebook.
- np.random.seed(1) is used to keep all the random function calls consistent. It will help us grade your work.

In [ ]:
import time
import numpy as np
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import dnn_app_utils_v3 as du
from skimage.transform import rescale, resize, downscale_local_mean

# refresh 'planar_utils' module
import imp
imp.reload(du)

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)
tf.random.set_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2 - Dataset

**Problem Statement**: You are given a dataset ("data.h5") containing:
    - a training set of m_train images labelled as cat (1) or non-cat (0)
    - a test set of m_test images labelled as cat and non-cat
    - each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB).

Let's get more familiar with the dataset. Load the data by running the cell below.

The training input and output are stored in 'train_x_orig' and 'train_y'. The test input and output are stored in 'test_x_orig' and 'test_y'. 

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes = du.load_data()
#print(classes)

The following code will show you an image in the dataset. Feel free to change the index and re-run the cell multiple times to see other images. 

y = 1. It's a cat picture.


In [ ]:
# Example of a picture
index = 200
plt.imshow(train_x_orig[index])
print("y = " + str(train_y[index,0]) + ". It's a " + classes[train_y[index,0]].decode("utf-8") +  " picture.")
#print(train_x_orig[index].shape,train_x_orig[index])

Let's explore more about the training data. We are curious about the shape of each image and can use 'train_x_orig[index].shape' to expore it. 
**Exercise**: Can you tell what the pixel number of each image is?

In [ ]:
# Explore your dataset 
image_shape=train_x_orig[index].shape
print("Each image is of size: ", image_shape)
#start your code here
num_px=209   # the number of pixels in each column of the image. Hint: The 1st element in image_shape indicates this number?
#end your code here
print('Number of pixels in each column of the image:', num_px)

Each image is of size:  (64, 64, 3)
Number of pixels in each column of the image: 209


**Expected result:**

Each image is of size:  (64, 64, 3)

Number of pixels in each column of the image: 209


We just take a close look at an image of the image set. Now, let's take a look at the whole training set. 

**Exercise**: The first 4 lines in the following cell print out the shape of the trining images. There are 209 images, and each image has 64 by 64 pixcels and GRB color. 

Please print out the shape of the test input (stored in `test_x_orig`) and output (stored in `test_y`) data, and tell how many images we have in the test data. 

In [ ]:
train_x_shape=train_x_orig.shape
m_train = train_x_shape[0]
print ("train_x_orig shape: ",train_x_shape)
print ("Number of training examples: " ,m_train)
# start your code here
test_x_shape= test_x_orig.shape
test_y_shape= test_y.shape
m_test = test_y_shape[0]
# end your code here

print ("test_x_orig shape: " ,test_x_shape)
print ("test_y shape: " , test_y_shape)
print ("Number of test examples: " ,m_test)

train_x_orig shape:  (209, 64, 64, 3)
Number of training examples:  209
test_x_orig shape:  (50, 64, 64, 3)
test_y shape:  (50, 1)
Number of test examples:  50


**Expected result:**

train_x_orig shape:  (209, 64, 64, 3)

Number of training examples:  209

test_x_orig shape:  (50, 64, 64, 3)

test_y shape:  (50, 1)

Number of test examples:  50 

As mentioned in our lecture, you reshape and standardize the images before feeding them to the network. The code is given in the cell below.

![alt text](https://drive.google.com/uc?id=1TVZDDvQUKuSMUqMtS7Ahessd3-H8bqq9)
<caption><center> <u>Figure 1</u>: Image to vector conversion. <br> </center></caption>

In [ ]:
# Reshape the training and test examples 
train_x_flatten = train_x_orig.reshape(train_x_shape[0], -1)   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1)

Data normalization can help us accelerate the training process. The RGB values of an image are integers in the range of [0,255]. We can normalize the training images to have the feature values between 0 and 1 by using `train_x = train_x_flatten/255.0`.

**Exercise**: We normalize the training images using `train_x = train_x_flatten/255.0` in the 1st line. Please normalize the test images to have feature values between 0 and 1.

In [ ]:
# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.0

#start your code here
test_x = test_x_flatten/255.0
#end your code here

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))
print ("train_y's shape: " + str(train_y.shape))
print ("test_y's shape: " + str(test_y.shape))

train_x's shape: (209, 12288)
test_x's shape: (50, 12288)
train_y's shape: (209, 1)
test_y's shape: (50, 1)


**Expected result:**

train_x's shape: (209, 12288)

test_x's shape: (50, 12288)

train_y's shape: (209, 1)

test_y's shape: (50, 1)

###3- Build your neural network ###
Now, we are going to use the skills in the first module to build our neural network. 

Excep the input layer, you can use 

`tf.keras.layers.Dense(20,activation='relu', 
                            kernel_initializer='glorot_uniform',
                            bias_initializer='zeros')`

to build the layers. In this example, we have 20 neurons in this layer, use 'relu' as the activation function, initialize our weight randomly using 'glorot_uniform', and our bias is initialized to be 'zeros'. 

First, we will need an input layer. How many neurons will you need in the input layer? Hint: how many elements does every training input have?

Second, we will build several hidden layers. You are free to choose the number of neurons in every hidden layer. 

Last, we will build the output layer. How many neurons will you need in the output layer? Which activation function is more suitale in this problem? Hint: how many elements does every training output have? What is the range of the training output?


In [ ]:
tf.random.set_seed(1) # we fixed the seed of the random number generator, so every time running this code, we can have the same results.

# start your code here
cat_model=tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(12288)), # how many elements are there in every training image?
    tf.keras.layers.Dense(10,activation='relu', # in layer 1, we will have 20 neurons and use 'relu' as the activation function
                            kernel_initializer='glorot_uniform', # we will randomly choose the initial weight using the 'glorot_uniform' method
                            bias_initializer='zeros'), # we will initialize all bias to be zeros.
    tf.keras.layers.Dense(15,activation='relu', # in layer 1, we will have 20 neurons and use 'relu' as the activation function
                            kernel_initializer='glorot_uniform', # we will randomly choose the initial weight using the 'glorot_uniform' method
                            bias_initializer='zeros'), # we will initialize all bias to be zeros.
    tf.keras.layers.Dense(20,activation='relu', # in layer 1, we will have 20 neurons and use 'relu' as the activation function
                            kernel_initializer='glorot_uniform', # we will randomly choose the initial weight using the 'glorot_uniform' method
                            bias_initializer='zeros'), # we will initialize all bias to be zeros.
    tf.keras.layers.Dense(25,activation='relu', # in layer 1, we will have 20 neurons and use 'relu' as the activation function
                            kernel_initializer='glorot_uniform', # we will randomly choose the initial weight using the 'glorot_uniform' method
                            bias_initializer='zeros'), # we will initialize all bias to be zeros.
    tf.keras.layers.Dense(30,activation='relu', # in layer 1, we will have 20 neurons and use 'relu' as the activation function
                            kernel_initializer='glorot_uniform', # we will randomly choose the initial weight using the 'glorot_uniform' method
                            bias_initializer='zeros'), # we will initialize all bias to be zeros.
    #please follow the example of layer 1 to add more hidden layers here. 

    # End adding hidden layers                    
    tf.keras.layers.Dense(1,activation='sigmoid', # how many elements are there in every training output
                            kernel_initializer='glorot_uniform',
                            bias_initializer='zeros'),
    
])
# end your code here
cat_model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_176 (Dense)           (None, 10)                122890    
                                                                 
 dense_177 (Dense)           (None, 15)                165       
                                                                 
 dense_178 (Dense)           (None, 20)                320       
                                                                 
 dense_179 (Dense)           (None, 25)                525       
                                                                 
 dense_180 (Dense)           (None, 30)                780       
                                                                 
 dense_181 (Dense)           (None, 1)                 31        
                                                                 
Total params: 124,711
Trainable params: 124,711
Non-t

Now, let's compile the model. When comiling your model, you will need to specify the optimizer, the loss function, and the metrics.

For optimizers, the most popular ones are 'Adam' and its derivatives. You can also try 'RMSprop' and 'SGD' to see which one works better for you. You might need adjust the learning rate to acceperate the training process. 

For loss functions, our problem is a two-class classification problem. The most popular loss function would be 'BinaryCrossentropy'. If this is a multiple-class classification problem, you probably want to try 'CategoricalCrossentropy' or 'SparseCategoricalCrossentropy'. 

For the metrics, we want to know the accuracy of our model. Therefore, I would recommend 'accuracy'. 

In [ ]:
# start your code here
cat_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),  # Optimizer. Please change 'None' to some meaningful learning rate which is usually <1
    # Loss function to minimize
    loss=tf.keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=['accuracy'],
                 )
# end your code here

It's time to train. 

We can plot training history. It can help us to adjust the parameters. 

If the training loss increases, we may want to decrease the `learning_rate`.

If the training loss has large vibration, we may want to decrease the `learning_rate`. 

If the training loss decreases too slowly, we need to increase `learning_rate`.

If the training loss decreases at a good speed, but ends up large, we may want to increase the epoch number. 

Epoch 1/13
7/7 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.7273
Epoch 2/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 3/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 4/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 5/13
7/7 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 6/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 7/13
7/7 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 8/13
7/7 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 9/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555
Epoch 10/13
7/7 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.6555

KeyError: ignored

In [ ]:
# start your code here
history = cat_model.fit(
    train_x, # input data
    train_y, # real output data
    epochs=13, # how many iterations do you want to train your model? Your data will be reused to train the model for the number of epochs. I trained 5 min for this model
)
# end your code here. 
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss','val_loss'])

### 4. test ###
Now that both models are trained, let us test their performance using our test dataset. The test dataset is never used to train the models, and reflects the acutual performance of the model when it is used in the general images.

To evaluate the model , we can use
`cat_model.evaluate(test_x,test_y)`.

**Exercise**: Please evaluate the model with regularization. 

In [ ]:
cat_model.evaluate(test_x,test_y)

2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.3400


[0.0, 0.3400000035762787]

The expected accuracy is higher than 0.7. If the accuracy is lower than 0.7, you may want to increase the epoch number, adjust the learning rate, and the regularizer weight, to achieve the goal. 

For your reference, I trained the model for 5 min to achieve that goal. 

If you have achieved the 0.7 goal, I would like to challenge you for the 0.8 accuracy for the test set. 